In [1]:
%load_ext autoreload
%autoreload 2

%load_ext memory_profiler

from vflow import Vset
import numpy as np

from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

from functools import partial

from numpy.testing import assert_equal

ModuleNotFoundError: No module named 'memory_profiler'

In [2]:
def make_train_test():
    X, y = make_classification(n_samples=1000, n_features=5)
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    return(X_train, X_test, y_train, y_test)

train_test_vset = Vset(name = 'train_test', vfuncs=[make_train_test for i in range(10)])
train_test_vset_lazy = Vset(name = 'train_test', vfuncs=[make_train_test for i in range(10)], lazy=True)

# non-lazy eval
X_train, _, _, _ = train_test_vset(n_out=4, keys=['X_train', 'X_test', 'y_train', 'y_test'])

X_trains = list(X_train.values())
print(len(X_trains)) # 10 datasets + 1 for __prev__
print(X_trains[0].shape)
print(X_trains[0])

NameError: name 'Vset' is not defined

In [3]:
# lazy eval
X_train, _, y_train, _ = train_test_vset_lazy(n_out=4, keys=['X_train', 'X_test', 'y_train', 'y_test'])

X_trains = list(X_train.values())
print(len(X_trains)) # 10 promises + 1 for __prev__
promise_X = X_trains[0]
print(promise_X) # the values are (unfulfilled) promises to call sep_dicts since n_out > 1

inner_promise_X = promise_X.args[0]
print(inner_promise_X.vfunc.vfunc) # the arg passed to sep_dicts is itself a promise to call make_train_test

NameError: name 'train_test_vset_lazy' is not defined

In [4]:
y_trains = list(y_train.values()) # y_train is also a dict of unfulfilled promises
promise_y = y_trains[0]
inner_promise_y = promise_y.args[0]
print(promise_y)

NameError: name 'y_train' is not defined

In [5]:
# calling the promise fulfills it and caches the value for future calls
promise_X()
print(f'promise_X called: {promise_X.called}')
print(f'promise_X call cached: {id(promise_X.value) == id(promise_X())}')
print(promise_X)

NameError: name 'promise_X' is not defined

In [6]:
# the inner promises are shared by corresponding values of X_train and y_train
print(f'promise_y called: {inner_promise_y.called}')

print(promise_y) # promise_y's first arg is now a fulfilled promise (fulfilled by the call promise_X())

print(f'promise_y call cached: {id(inner_promise_y.value) == id(inner_promise_y())}')

NameError: name 'inner_promise_y' is not defined